# 📖 Bible Verse Theme Tagger

This notebook automatically tags 31,103 Bible verses with contextual themes using Google Gemini AI.

## What it does:
- Reads verses from your Bible database
- Uses Gemini AI to analyze context and sentiment
- Tags each verse with relevant themes (anxiety, comfort, hope, strength, etc.)
- **Updates ONLY the themes column** (doesn't replace your database)

## Setup Required:
1. Upload `bible.db` from assets/bible.db
2. Add your Gemini API key below
3. Click Runtime → Run All
4. Download the updated `bible.db` when complete
5. Replace the file in assets/bible.db

**Estimated time:** 20-30 minutes for 31k verses

## Step 1: Install Dependencies

In [ ]:
!pip install -q google-generativeai pandas tqdm

## Step 2: Configure Gemini API

In [ ]:
import google.generativeai as genai
import pandas as pd
import json
import time
from tqdm import tqdm
import re
import sqlite3
import shutil

# 🔑 ADD YOUR GEMINI API KEY HERE
GEMINI_API_KEY = ""  # Get from https://makersuite.google.com/app/apikey

if not GEMINI_API_KEY:
    raise ValueError("⚠️ Please add your Gemini API key above!")

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash-exp')

print("✅ Gemini AI configured successfully!")

## Step 3: Define Biblical Themes

These are the themes Gemini will choose from when tagging verses.

In [ ]:
BIBLICAL_THEMES = [
    # Emotional/Spiritual States
    "anxiety", "peace", "fear", "courage", "hope", "despair",
    "joy", "sadness", "anger", "forgiveness", "gratitude",
    "loneliness", "comfort", "depression", "doubt", "faith",
    
    # Spiritual Themes
    "prayer", "worship", "praise", "repentance", "salvation",
    "redemption", "grace", "mercy", "love", "trust",
    
    # Life Situations
    "guidance", "wisdom", "strength", "weakness", "suffering",
    "persecution", "trials", "perseverance", "patience",
    "provision", "protection", "healing", "sickness",
    
    # Relationships
    "marriage", "family", "friendship", "unity", "conflict",
    "reconciliation", "humility", "pride",
    
    # Purpose & Identity
    "purpose", "calling", "identity", "worth", "obedience",
    "righteousness", "holiness", "transformation",
    
    # God's Attributes
    "faithfulness", "sovereignty", "presence", "power",
    "justice", "compassion", "eternal", "creator"
]

print(f"📋 Using {len(BIBLICAL_THEMES)} biblical themes for tagging")
print(f"Themes: {', '.join(BIBLICAL_THEMES[:10])}...")

## Step 4: Upload Bible Database

Upload your `bible.db` file from `assets/bible.db`

In [ ]:
from google.colab import files

print("📤 Upload your bible.db file from assets/bible.db...")
uploaded = files.upload()

# Make a backup copy
shutil.copy('bible.db', 'bible_backup.db')
print("\n✅ Backup created: bible_backup.db")

# Load verses from database
conn = sqlite3.connect('bible.db')
df = pd.read_sql_query(
    "SELECT id, book, chapter, verse_number, text, translation, reference FROM verses WHERE translation='WEB'",
    conn
)
conn.close()

print(f"\n✅ Loaded {len(df):,} verses from database")
print(f"\nSample verse:")
print(df.iloc[0])

## Step 5: Clean Verse Text

Remove Strong's numbers and formatting artifacts for cleaner AI analysis.

In [ ]:
def clean_verse_text(text):
    """Remove Strong's numbers and artifacts from verse text"""
    if pd.isna(text):
        return ""
    
    # Remove Strong's numbers: |strong="H1234" or strong="G5678"
    text = re.sub(r'\|strong="[HG]\d+"', '', text)
    
    # Remove WH+ markup: +wh text+wh*
    text = re.sub(r'\+wh[^\+]*\+wh\*', '', text)
    
    # Remove extra pipes and normalize whitespace
    text = text.replace('|', ' ')
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Add cleaned text column
df['clean_text'] = df['text'].apply(clean_verse_text)

print("✅ Cleaned verse text")
print(f"\nOriginal: {df.iloc[0]['text'][:100]}...")
print(f"Cleaned:  {df.iloc[0]['clean_text'][:100]}...")

## Step 6: Tag Verses with Gemini AI

This is the main processing step. It will:
- Process verses in batches of 20 (smaller for better accuracy)
- Use Gemini to analyze context and sentiment
- Assign 1-5 relevant themes per verse
- Handle rate limits with automatic retries

**⏱️ This will take 20-30 minutes. You can leave the tab open.**

In [ ]:
def tag_verse_batch(verses_batch):
    """Tag a batch of verses with themes using Gemini"""
    
    # Build prompt with all verses in batch
    prompt = f"""You are a biblical scholar analyzing verses for thematic content.

For each verse below, identify 1-5 relevant themes from this list:
{', '.join(BIBLICAL_THEMES)}

Consider:
- The emotional tone and sentiment
- The spiritual message or lesson
- The practical application for readers
- The context of suffering, joy, guidance, etc.

Return ONLY a JSON array where each element is an array of theme strings.
Example: [["hope", "faith"], ["anxiety", "peace", "trust"], ["wisdom", "guidance"]]

Verses to tag:
"""
    
    for idx, verse in enumerate(verses_batch):
        prompt += f"\n{idx+1}. {verse['reference']}: {verse['clean_text'][:200]}"
    
    prompt += "\n\nJSON array of themes:"
    
    try:
        response = model.generate_content(prompt)
        
        # Extract JSON from response
        response_text = response.text.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith('```'):
            response_text = re.sub(r'```json\s*|```\s*', '', response_text)
        
        themes_batch = json.loads(response_text)
        
        if len(themes_batch) != len(verses_batch):
            print(f"⚠️ Mismatch: Expected {len(verses_batch)} but got {len(themes_batch)}")
            return None
        
        return themes_batch
        
    except Exception as e:
        print(f"❌ Error processing batch: {e}")
        return None

# Initialize themes column
df['themes'] = None

# Process in smaller batches for better accuracy
BATCH_SIZE = 20
total_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE

print(f"🚀 Starting to tag {len(df):,} verses in {total_batches} batches...\n")

for batch_idx in tqdm(range(0, len(df), BATCH_SIZE), desc="Tagging verses"):
    batch_df = df.iloc[batch_idx:batch_idx + BATCH_SIZE]
    
    verses_batch = batch_df[['reference', 'clean_text']].to_dict('records')
    
    # Tag the batch with retry logic
    themes_batch = None
    for retry in range(3):
        themes_batch = tag_verse_batch(verses_batch)
        if themes_batch is not None:
            break
        print(f"Retry {retry+1}/3 for batch {batch_idx//BATCH_SIZE + 1}...")
        time.sleep(2)
    
    if themes_batch is None:
        print(f"⚠️ Skipping batch {batch_idx//BATCH_SIZE + 1} after 3 retries")
        continue
    
    # Store themes as JSON strings
    for i, themes in enumerate(themes_batch):
        df.at[batch_idx + i, 'themes'] = json.dumps(themes)
    
    # Rate limiting: ~60 requests per minute
    time.sleep(1)

print("\n✅ Tagging complete!")
print(f"Tagged verses: {df['themes'].notna().sum():,}")
print(f"Missing tags: {df['themes'].isna().sum():,}")

## Step 7: Preview Tagged Verses

In [ ]:
# Show sample tagged verses
sample_verses = df[df['themes'].notna()].sample(min(10, len(df)))

print("📖 Sample Tagged Verses:\n")
for _, verse in sample_verses.iterrows():
    themes = json.loads(verse['themes'])
    print(f"{verse['reference']}")
    print(f"Text: {verse['clean_text'][:100]}...")
    print(f"Themes: {', '.join(themes)}")
    print()

## Step 8: Update Database with Themes

This updates ONLY the themes column in the existing database.

In [ ]:
# Update the database with themes
conn = sqlite3.connect('bible.db')
cursor = conn.cursor()

print("📝 Updating database with themes...\n")

for _, row in tqdm(df.iterrows(), total=len(df), desc="Updating themes"):
    if pd.notna(row['themes']):
        cursor.execute(
            "UPDATE verses SET themes = ? WHERE id = ?",
            (row['themes'], row['id'])
        )

conn.commit()
conn.close()

print("\n✅ Database updated successfully!")

# Verify the update
conn = sqlite3.connect('bible.db')
result = pd.read_sql_query(
    "SELECT COUNT(*) as total, SUM(CASE WHEN themes IS NOT NULL THEN 1 ELSE 0 END) as tagged FROM verses WHERE translation='WEB'",
    conn
)
conn.close()

print(f"\n📊 Database Statistics:")
print(f"  - Total verses: {result['total'].iloc[0]:,}")
print(f"  - Tagged verses: {result['tagged'].iloc[0]:,}")
print(f"  - Coverage: {(result['tagged'].iloc[0] / result['total'].iloc[0] * 100):.1f}%")

## Step 9: Download Updated Database

In [ ]:
print("📥 Downloading updated bible.db...")
files.download('bible.db')

print("\n🎉 ALL DONE!")
print("\nNext steps:")
print("1. Replace assets/bible.db with the downloaded file")
print("2. Update ai_provider.dart to use searchByTheme() instead of searchVerses()")
print("3. Test the AI chat to see diverse, contextually relevant verses!")
print("\n💡 The backup (bible_backup.db) is also available if needed.")